In [1]:
import pandas as pd
import numpy as np
from utils import get_relation

C:\Users\i5\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\i5\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\i5\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_excel("test_data.xlsx")

In [3]:
df

,121_col1,121_col2,12m_col1,12m_col2,m2m_col1,m2m_col2,121_col1_int,121_col2_int
0,A1,A1,B1,B11,D1,D1,1,1.0
1,A2,A2,B1,B12,D2,D1,2,2.0
2,A2,A2,B1,B13,D3,D1,2,2.0
3,A3,NaN,B2,B21,D1,D2,3,NaN
4,A3,A3,B2,B21,D2,D2,3,3.0
5,A4,A4,B2,B22,D3,D2,4,4.0


In [4]:
df

,121_col1,121_col2,12m_col1,12m_col2,m2m_col1,m2m_col2,121_col1_int,121_col2_int
0,A1,A1,B1,B11,D1,D1,1,1.0
1,A2,A2,B1,B12,D2,D1,2,2.0
2,A2,A2,B1,B13,D3,D1,2,2.0
3,A3,NaN,B2,B21,D1,D2,3,NaN
4,A3,A3,B2,B21,D2,D2,3,3.0
5,A4,A4,B2,B22,D3,D2,4,4.0


In [5]:
col1 = "121_col1"
col2 = "121_col2"



df.get_relation(col1, col2, ignore_na = True)
df.get_relation(col1, col2, ignore_na = True)

'one-to-one'

In [11]:

print("gb col1", df[[col1, col2]].groupby(col1).nunique().max())
print("gb col2", df[[col1, col2]].groupby(col2).nunique().max())
print(get_relation(df, col1, col2), ' reversed ', get_relation(df, col2, col1))
print(get_relation(df, col1, col2, ignore_na=False), ' reversed ', get_relation(df, col2, col1, ignore_na=False))

gb col1 121_col2    2
dtype: int64
gb col2 121_col1    1
dtype: int64
one-to-many  reversed  many-to-one
one-to-many  reversed  many-to-one


In [49]:
col1 = "121_col1_int"
col2 = "121_col2_int"
print("gb col1", df[[col1, col2]].groupby(col1).nunique().max())
print("gb col2", df[[col1, col2]].groupby(col2).nunique().max())
print(get_relation(df, col1, col2), ' reversed ', get_relation(df, col2, col1))
print("use na", get_relation(df, col1, col2, ignore_na=False), ' reversed ', get_relation(df, col2, col1, ignore_na=False))

gb col1 121_col2_int    1
dtype: int64
gb col2 121_col1_int    1
dtype: int64
one-to-one  reversed  one-to-one
use na one-to-many  reversed  many-to-one


In [50]:
col1 = "12m_col1"
col2 = "12m_col2"
print("gb col1", df[[col1, col2]].groupby(col1).nunique().max())
print("gb col2", df[[col1, col2]].groupby(col2).nunique().max())
print(get_relation(df, col1, col2), ' reversed ', get_relation(df, col2, col1))

gb col1 12m_col2    3
dtype: int64
gb col2 12m_col1    1
dtype: int64
one-to-many  reversed  many-to-one


In [51]:
col1 = "m2m_col1"
col2 = "m2m_col2"
print("gb col1", df[[col1, col2]].groupby(col1).nunique().max())
print("gb col2", df[[col1, col2]].groupby(col2).nunique().max())
print(get_relation(df, col1, col2), ' reversed ', get_relation(df, col2, col1))

gb col1 m2m_col2    2
dtype: int64
gb col2 m2m_col1    3
dtype: int64
many-to-many  reversed  many-to-many


## Num unique pairings

In [62]:
def maximum_unique_mapping(df, col1, col2):
    """returns the maximum number of unique values in col2 that map to a single value in col1
    So col1, col2
       A, duck
       A, goat
       B, chicken
       
    would yield a value of 2, as both duck and goat map to a single value in column 1, and that number (2) is 
    higher than the single value (chicken) that maps to B"""
    
    max_unique_values = pd.pivot_table(df.drop_duplicates([col1, col2]), index=[col1], aggfunc="count", values=col2).max()
    return int(max_unique_values)

def maximum_unique_mappings(df, ignore_cols=[]):
    """Returns an n x n dataframe where n = len(df.columns). The values in each cell are the maximum number of values that
map to a single value in another column. The index columns being the reference"""
    
    cols_to_use = [col for col in list(df.columns) if col not in ignore_cols]
    blank_df = pd.DataFrame(index = cols_to_use, columns = cols_to_use)
    for col1 in cols_to_use:
        for col2 in cols_to_use:
            if col1 == col2:
                continue
            blank_df.loc[col1, col2] = maximum_unique_mapping(df, col1, col2)
    return blank_df

In [76]:
maximum_unique_mappings(df)

,121_col1,121_col2,12m_col1,12m_col2,m2m_col1,m2m_col2,121_col1_int,121_col2_int
121_col1,NaN,1,1,2,2,1,1,1
121_col2,1,NaN,1,2,2,1,1,1
12m_col1,2,2,NaN,3,3,1,2,2
12m_col2,1,1,1,NaN,2,1,1,1
m2m_col1,2,2,2,2,NaN,2,2,2
m2m_col2,2,2,1,3,3,NaN,2,2
121_col1_int,1,1,1,2,2,1,NaN,1
121_col2_int,1,1,1,2,2,1,1,NaN


In [84]:
df.shape[0]

6

In [85]:
col1 = "121_col1"
col2 = "121_col2"
maximum_unique_mapping(df.fillna("-1"), col1, col2)

2